In [1]:
import numpy as np
import numpy.typing as npt
from numba import jit, njit, prange
import pickle
from dataclasses import dataclass, asdict, field
import argparse
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd
from pathlib import Path
from typing import Any
import time
import sys
import copy
import concurrent.futures as cf

from src.dataclass import (
    Input, Lattice, Parameter, Train, Save,
    Processed_Input, Topology, Conjugate, Result
)
from src.manage_data import save_result, save_log, load_result
from src.initial_state import get_initial_state
from src.process_input import (get_processed_input, get_T_and_H, get_J)
from src.metropolis import execute_metropolis_update
from src.function import (time_correlation, column_average_2d, get_spin_glass)
from src.process_output import get_result

In [2]:
test = np.array([[1, 2, 3],[4, 5, 6]])
list = np.array([0,1])
print(test[:,list])

[[1 2]
 [4 5]]


In [8]:
@njit(parallel=True)
def sigma_i_sigma_j(array, measurement, length):
    avg = np.zeros(length)
    corr = np.zeros((length, length))

    for i in prange(length):
        for j in prange(measurement):
            avg[i] = avg[i] + array[j, i]

    avg = avg / measurement

    for i in prange(length):
        for j in prange(length):
            for k in prange(measurement):
                corr[i][j] = corr[i][j] + \
                    np.conjugate(array[k, i]) * array[k, j]

    corr = corr / measurement

    for i in prange(length):
        for j in prange(length):
            corr[i][j] = corr[i][j] - avg[i] * avg[j]

    return np.real(corr)

def space_correlation(
    array: npt.NDArray,
) -> npt.NDArray:
    """
    array: [measurement, size**dim]
    return: [size**dim, size**dim]
    """
    measurement = np.size(array[:, 0])
    length = np.size(array[0])

    average = np.einsum("ij->j", array, optimize=True) / measurement
    corr = np.tensordot(np.conjugate(array), array, (0, 0)) / measurement

    return np.real(corr - np.tensordot(np.conjugate(average), average, axes=0))

In [12]:
measurement, length = 10000, 1024
test = np.random.rand(measurement, length)

In [13]:
# %timeit sigma_i_sigma_j(test, measurement, length)

In [ ]:
# %timeit space_correlation(test)

10.3 ms ± 8.22 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

"""
Memory
Max(measurement * size**dim, size**(2*dim)) * max_workers

Performance
measurement * size**dim * ensembles / max_workers
"""

"""
Lattice Condition
"""
args.state = 3
args.size = 2**6
args.dimension = 2
args.ghost = 0
args.initial = "uniform" # "uniform", "random"

"""
Parameter Condition
"""
args.T, args.H = 1.0, 0.0
# q=2: 2.2692, q=3: 1.4925
# args.Tc = 1/((1-1/args.state)*np.log(1+np.sqrt(args.state)))
args.Tc = 1.5
args.Hc = 0.0
args.Jm = 1.0
args.Jv = 1.0

args.mode = "normal"  # 'normal', 'critical' and 'manual'
args.variable = "T"  # 'T', 'H'
args.multiply = 0.1**4
args.base = 2.0
args.exponent = 13

"""
Train Condition
"""
args.iteration = 2**10
args.measurement = 2**14
args.interval = 2**3
args.sweep = args.measurement * args.interval
args.ensemble = 2**3
args.max_workers = 2**3
args.threshold = 5 * 0.1**2
args.recent = 10**2

"""
Save Condition
"""
args.environment = "server" # "server" or "local"
args.location = "result"
args.save = True  # True or False

lattice = Lattice(
args.state, args.size, args.dimension, 
args.ghost, args.initial)
parameter = Parameter(
args.Tc, args.Hc, args.Jm, args.Jv, args.mode, 
args.variable, args.multiply, args.base, args.exponent)
train = Train(
args.iteration, args.sweep, args.measurement, args.interval,
args.ensemble, args.max_workers, args.threshold, args.recent)
save = Save(args.environment, args.location, args.save)

input = Input(lattice, parameter, train, save)
input.parameter.T, input.parameter.H = args.T, args.H

processed_input = get_processed_input(input)
input.parameter.T, input.parameter.H = get_T_and_H(input)
J = get_J(input, processed_input)
print(input)

Input(lattice=Lattice(state=3, size=64, dimension=2, ghost=0, initial='uniform'), parameter=Parameter(T=2.3192000000000004, H=0.0, Tc=1.5, Hc=0.0, Jm=1.0, Jv=1.0, mode='normal', variable='T', multiply=0.00010000000000000002, base=2.0, exponent=13), train=Train(iteration=1024, sweep=131072, measurement=16384, interval=8, ensemble=8, max_workers=8, threshold=0.05000000000000001, recent=100), save=Save(environment='server', save=True))


In [ ]:
def execute_metropolis_update(
    input: Input, processed_input: Processed_Input, J: npt.NDArray[np.float64], system_state: npt.NDArray[np.complex128],
) -> npt.NDArray[np.complex128]:
    lattice, parameter, topology, conjugate = (
        input.lattice,
        input.parameter,
        processed_input.topology,
        processed_input.conjugate
    )

    size, dimension, state, T, H, interaction_point, complex_state, conjugate_state, complex_ghost = (
        lattice.size,
        lattice.dimension,
        lattice.state,
        parameter.T,
        parameter.H,
        topology.interaction_point,
        conjugate.complex_state,
        conjugate.conjugate_state,
        conjugate.complex_ghost,
    )

    rng = np.random.default_rng()
    flip_coord = np.arange(size**dimension)
    rng.shuffle(flip_coord)
    prob = rng.random(size=size**dimension)
    idx = rng.integers(state-1, size=size**dimension)
    
    return update_system_state(flip_coord, system_state, prob, idx, state, H, T, complex_ghost, complex_state, conjugate_state, J, interaction_point)


@njit
def update_system_state(flip_coord, system_state, prob, idx, state, H, T, complex_ghost, complex_state, conjugate_state, J, interaction_point):

    for i, x in enumerate(flip_coord):
        index_list = np.arange(state)
        angle = np.int64(np.round((np.angle(system_state[x])/2/np.pi*state)%state))
        index_list = np.delete(index_list, angle)
        # rng = np.random.default_rng()
        proposal = index_list[idx[i]]

        current_energy, flip_energy = 0, 0

        interaction = H * complex_ghost
        for point in interaction_point[x]:
            interaction += J[x][point] * system_state[point]

        current_energy = np.real(- conjugate_state[angle] * interaction)
        flip_energy = np.real(- conjugate_state[proposal] * interaction)
        
        if flip_energy <= current_energy:
            system_state[x] = complex_state[proposal]
        
        else:
            if(prob[i] <= np.exp(- (flip_energy - current_energy) / T)):
                system_state[x] = complex_state[proposal]

    return system_state

In [ ]:
print(input)
update = get_initial_state(input)
iter = 1000
result = np.empty((iter+1, input.lattice.size**input.lattice.dimension), dtype=np.complex128)
result[0] = update
for i in range(iter):
    update = execute_metropolis_update(input, processed_input, J, update)
    result[i+1] = update
print(np.einsum("ij->i", result)/input.lattice.size**input.lattice.dimension)
print(np.einsum("ij->", result)/input.lattice.size**input.lattice.dimension/(iter+1))
%timeit execute_metropolis_update(input, processed_input, J, update)

In [ ]:
print(processed_input.conjugate)